In [2]:
import numpy as np
import pandas as pd

# 第六章 缺失数据

#### (c)  NaT
##### NaT是针对时间序列的缺失值，是Pandas的内置类型，可以完全看做时序版本的np.nan，与自己不等且使用equals是也会被跳过

In [2]:
s_time = pd.Series([pd.Timestamp('20120101')]*5)
s_time

0   2012-01-01
1   2012-01-01
2   2012-01-01
3   2012-01-01
4   2012-01-01
dtype: datetime64[ns]

In [3]:
s_time[2] = None
s_time

0   2012-01-01
1   2012-01-01
2          NaT
3   2012-01-01
4   2012-01-01
dtype: datetime64[ns]

In [5]:
s_time[3] = np.nan
s_time

0   2012-01-01
1   2012-01-01
2          NaT
3          NaT
4   2012-01-01
dtype: datetime64[ns]

In [7]:
s_time[2] = pd.NaT
s_time

0   2012-01-01
1   2012-01-01
2          NaT
3          NaT
4   2012-01-01
dtype: datetime64[ns]

In [8]:
type(s_time[2])

pandas._libs.tslibs.nattype.NaTType

In [10]:
s_time[2] == s_time[3]

False

In [11]:
s_time.equals(s_time)

True

In [12]:
s = pd.Series([True,False],dtype='bool')
s[1] = pd.NaT
s

0    True
1    True
dtype: bool

### 3.Nullable类型与NA符号

##### 这是Pandas在1.0新版本中引入的重大改变，其目的就是为了（在若干版本后）解决之前出现的混乱局面，统一缺失值处理方法

#### (a)Nullable整型

##### 对于该种类型而言，它与原来标记int上的符号区别在于首字母大写：‘Int’

In [14]:
s_original = pd.Series([1,2],dtype = 'int64')
s_original

0    1
1    2
dtype: int64

In [15]:
s_new = pd.Series([1,2],dtype = 'Int64')
s_new

0    1
1    2
dtype: Int64

##### 它的好处就在于，其中前面提到的三种缺失值都会被替换为统一的NA符号，且不改变数据类型

In [16]:
s_original[1] = np.nan
s_original

0    1.0
1    NaN
dtype: float64

In [4]:
s_new[1] = np.nan
s_new

0    False
1     <NA>
dtype: boolean

In [5]:
s_new[1] = np.nan
s_new                 

0    False
1     <NA>
dtype: boolean

In [6]:
s_new[1] = pd.NaT
s_new  

0    False
1     <NA>
dtype: boolean

#### (b) Nullable布尔
##### 对于该种类型而言，作用与上面的类似，记号为boolean

In [7]:
s_original = pd.Series([1,0],dtype='bool')
s_original

0     True
1    False
dtype: bool

In [15]:
s_new = pd.Series([0,1],dtype = "boolean")
s_new

0    False
1     True
dtype: boolean

In [8]:
s_original[1] = np.nan
s_original

0    1.0
1    NaN
dtype: float64

In [10]:
s_original = pd.Series([1, 0], dtype="bool") #此处重新加一句是因为前面赋值改变了bool类型
s_original[0] = None
s_original

0    False
1    False
dtype: bool

In [16]:
s_new[0] = np.nan
s_new

0    <NA>
1    True
dtype: boolean

In [17]:
s_new[0] = None
s_new

0    <NA>
1    True
dtype: boolean

In [18]:
s_new[0] = pd.NaT
s_new

0    <NA>
1    True
dtype: boolean

##### 需要注意的是，含有pd.NA的布尔列表在1.0.2之前的版本作为索引时会报错，这是一个之前的bug，现已经修复

In [19]:
s = pd.Series(['dog','cat'])
s[s_new]

1    cat
dtype: object

#### (c) string类型

#####  该类型是1.0的一大创新，目的之一就是为了区分开原本含糊不清的object类型，这里将简要地提及string，因为它是第7章的主题内容
##### 它本质上也属于Nullable类型，因为并不会因为含有缺失而改变类型

In [20]:
s = pd.Series(['dog','cat'],dtype = 'string')
s

0    dog
1    cat
dtype: string

In [21]:
s[0] = np.nan
s

0    <NA>
1     cat
dtype: string

In [22]:
s[0] = None
s

0    <NA>
1     cat
dtype: string

In [23]:
s[0] = pd.NaT
s

0    <NA>
1     cat
dtype: string

##### 此外，和object类型的一点重要区别就在于，在调用字符方法后，string类型返回的是Nullable类型，object则会根据缺失类型和数据类型而改变

In [24]:
s = pd.Series(['a',None,'b'],dtype = 'string')
s

0       a
1    <NA>
2       b
dtype: string

In [25]:
s.str.count('a')

0       1
1    <NA>
2       0
dtype: Int64

In [26]:
s2 = pd.Series(['a',None,'b'],dtype = 'object')
s2

0       a
1    None
2       b
dtype: object

In [27]:
s2.str.count('a')

0    1.0
1    NaN
2    0.0
dtype: float64

In [28]:
s.str.isdigit()

0    False
1     <NA>
2    False
dtype: boolean

### 4.NA的特性
#### （a） 逻辑运算
##### 只需看该逻辑运算的结果是否依赖pd.NA的取值，如果依赖，则结果还是NA，如果不依赖，则直接计算结果

In [29]:
True | pd.NA

True

In [30]:
pd.NA | True

True

In [31]:
False | pd.NA

<NA>

In [32]:
True & pd.NA

<NA>

##### 取值不明直接报错

In [33]:
bool(pd.NA)

TypeError: boolean value of NA is ambiguous

### (b) 算术运算和比较运算
##### 这里只需要记住除了下面两种情况，其他结果都是NA即可

In [34]:
pd.NA ** 0

1

In [35]:
1 ** pd.NA

1

##### 其他情况

In [36]:
pd.NA + 1 

<NA>

In [37]:
'a' * pd.NA

<NA>

In [38]:
pd.NA == pd.NA

<NA>

In [39]:
np.nan == np.nan

False

In [40]:
pd.NA < 2.5

<NA>

In [41]:
np.log(pd.NA)

<NA>

In [42]:
np.add(pd.NA,1)

<NA>

### 5.convert_dtypes方法
##### 这个函数的功能往往就是在读取数据时，就把数据列转为Nullable类型，是1.0的新函数

In [47]:
pd.read_csv(r'C:\Users\gan\Desktop\joyful-pandas\data\table_missing.csv').dtypes

School      object
Class       object
ID         float64
Gender      object
Address     object
Height       int64
Weight     float64
Math       float64
Physics     object
dtype: object

In [48]:
pd.read_csv(r'C:\Users\gan\Desktop\joyful-pandas\data\table_missing.csv').convert_dtypes().dtypes

School      string
Class       string
ID           Int64
Gender      string
Address     string
Height       Int64
Weight       Int64
Math       float64
Physics     string
dtype: object

## 二、缺失数据的运算和分组
#### 1. 加号与乘号规则
##### 使用加法时，缺失值为0

In [49]:
s = pd.Series([2,3,np.nan,4])
s.sum()

9.0

#####  使用乘法时，缺失值为1

In [50]:
s.prod()

24.0

##### 使用累计函数时，缺失值自动略过

In [51]:
s.cumsum()

0    2.0
1    5.0
2    NaN
3    9.0
dtype: float64

In [52]:
s.cumprod()

0     2.0
1     6.0
2     NaN
3    24.0
dtype: float64

In [53]:
s.pct_change()

0         NaN
1    0.500000
2    0.000000
3    0.333333
dtype: float64

### 2. groupby方法中的缺失值
##### 自动忽略为缺失值的组

In [54]:
df_g = pd.DataFrame({'one':['A','B','C','D',np.nan],'two':np.random.randn(5)})
df_g

,one,two
0,A,-0.949064
1,B,-0.984033
2,C,-0.269458
3,D,-0.982695
4,NaN,-1.517009


In [55]:
df_g.groupby('one').groups

{'A': Int64Index([0], dtype='int64'),
 'B': Int64Index([1], dtype='int64'),
 'C': Int64Index([2], dtype='int64'),
 'D': Int64Index([3], dtype='int64')}